<a href="https://colab.research.google.com/github/vpuzio-deloitte/digita/blob/main/digita_image_extraction_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install langchain_core langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.7 MB/s eta 0:00:00


In [7]:
fruits = ['https://immagini.editorialedomani.it/version/c:ZmE4YzhhMWItZTE2OS00:ZTgzMTAzMGEtYWQ4MC00/75th-sanremo-song-festival.webp?f=3%3A2&q=0.75&w=635',
          'https://storage.googleapis.com/vectrix-public/fruit/apple.jpeg',
          'https://storage.googleapis.com/vectrix-public/fruit/banana.jpeg',
          'https://storage.googleapis.com/vectrix-public/fruit/kiwi.jpeg',
          'https://storage.googleapis.com/vectrix-public/fruit/peach.jpeg',
          'https://storage.googleapis.com/vectrix-public/fruit/plum.jpeg']

In [9]:
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
import base64, httpx
from google.colab import userdata
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


# Initialize the model
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=userdata.get('GOOGLE_API_KEY'))

# Download and encode the image
image_data = base64.b64encode(httpx.get(fruits[0]).content).decode("utf-8")

# Create a message with the image
message = HumanMessage(
    content=[
        {"type": "text", "text": """
        sei uno spettatore di sanremo, il classico italiano medio, un po' rozzo

        descrivimi la scena

        in romanesco
        """},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
        },
    ],
)

# Invoke the model with the message
response = model.invoke([message])

# Print the model's response
print(response.content)

Aoh, che casino!  Stiamo a Sanremo, eh?  'Na luce accecante, sembra 'na discoteca de lusso, ma con più velluto e meno sudore.  

Quella lì in mezzo, tutta brillantini, pare 'na palla da discoteca che ha trovato 'n vestito.  Che voce, però...se la tira un po', eh, ma vabbè...  'na gran figa, 'n'è che si possa dì.

Ai lati, due vecchietti eleganti, ma che sembrano usciti da 'na pubblicità de funerali.  Uno de sti due, quello più grasso,  sembra che se deve appoggià a 'na sedia per non crollà.  L'altro, quello più magro,  pare 'n avvocato che ha appena perso 'na causa milionaria.  

Ma la musica?  Ah, la musica!  Uno sbrodo di note,  'n miscuglio de pop,  rap e chissà che altro...  Ma se la cantasse mia nonna, sarebbe meglio.  

Comunque, 'na serata da trascorrere col telecomando in mano,  pronti a cambiare canale se passa qualcosa de meglio...  magari 'na pubblicità de birra.  
